In [ ]:
import pandas as pd
from google.colab import drive
import os

drive.mount('/content/drive')

archivo_dashboard = "/content/drive/MyDrive/AUDITORIA/DASHBOARD AGOSTO 2025.xlsx"
archivo_dispersiones = "/content/drive/MyDrive/AUDITORIA/DISPERSIONES AGOSTO 2025.xlsx"

df_dispersiones = pd.read_excel(archivo_dispersiones)
df_dispersiones.columns = df_dispersiones.columns.str.strip()

print("Columns in df_dispersiones after stripping:", df_dispersiones.columns)

# === VALIDAR COLUMNAS REQUERIDAS ===
for col in ['ENTIDAD', 'ID', 'VALOR', 'RETENCION', 'DISPERSION']:
    if col not in df_dispersiones.columns:
        raise KeyError(f"La columna '{col}' no se encuentra en dispersiones.")

# === LIMPIEZA DE DISPERSIONES ===
df_dispersiones['ENTIDAD'] = df_dispersiones['ENTIDAD'].astype(str).str.strip().str.upper()

df_dispersiones['ID'] = (
    df_dispersiones['ID'].astype(str)
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.replace(",", "", regex=False)
)
df_dispersiones['VALOR'] = (
    df_dispersiones['VALOR'].astype(str)
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
)
df_dispersiones['VALOR'] = pd.to_numeric(df_dispersiones['VALOR'], errors='coerce').fillna(0)

df_dispersiones['RETENCION'] = (
    df_dispersiones['RETENCION'].astype(str)
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.replace(",", "", regex=False)
)
df_dispersiones['RETENCION'] = pd.to_numeric(df_dispersiones['RETENCION'], errors='coerce').fillna(0)

df_dispersiones['DISPERSION'] = (
    df_dispersiones['DISPERSION'].astype(str)
    .str.strip()
    .str.replace(" ", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.replace(",", "", regex=False)
)
df_dispersiones['DISPERSION'] = pd.to_numeric(df_dispersiones['DISPERSION'], errors='coerce').fillna(0)

# === FUNCIÓN PARA AUDITAR ===
def auditar_archivo(ruta_archivo, df_base):
    nombre = os.path.basename(ruta_archivo).split('.')[0]
    print(f"\n🔍 Auditando: {nombre}")

    try:
        df = pd.read_excel(ruta_archivo)
        df.columns = df.columns.str.strip()

        for col in ['ENTIDAD', 'ID', 'VALOR', 'RETENCION', 'DISPERSION']:
            if col not in df.columns:
                print(f"{nombre}: No se encuentran las columnas '{col}'.")
                return {}, pd.DataFrame()

        # === LIMPIEZA DE DATOS DEL DASHBOARD ===
        df['ENTIDAD'] = df['ENTIDAD'].astype(str).str.strip().str.upper()
        df['ID'] = (
            df['ID'].astype(str)
            .str.strip()
            .str.replace(" ", "", regex=False)
            .str.replace(".", "", regex=False)
            .str.replace(",", "", regex=False)
        )
        df['VALOR'] = (
            df['VALOR'].astype(str)
            .str.strip()
            .str.replace(" ", "", regex=False)
            .str.replace(".", "", regex=False)
            .str.replace(",", ".", regex=False)
        )
        df['VALOR'] = pd.to_numeric(df['VALOR'], errors='coerce').fillna(0)

        # === COMPARACIÓN POR COMERCIO + IdTransaccion ===
        comparacion = pd.merge(
            df_base[['ENTIDAD', 'ID', 'VALOR', 'RETENCION', 'DISPERSION']],
            df[['ENTIDAD', 'ID', 'VALOR', 'RETENCION', 'DISPERSION']],
            left_on=['ENTIDAD', 'ID'],
            right_on=['ENTIDAD', 'ID'],
            how='outer',
            suffixes=('_dispersiones', '_dashboard'),
            indicator=True
        )

        # === CLASIFICAR RESULTADOS ===
        comparacion['resultado'] = comparacion.apply(
            lambda r: (
                'Coincide'
                if abs(r['VALOR_dispersiones'] - r['VALOR_dashboard']) < 1
                else 'Falta en dispersiones'
                if pd.isna(r['VALOR_dispersiones'])
                else 'Falta en dashboard'
            ),
            axis=1
        )

        # === SEPARAR RESULTADOS ===
        coinciden = comparacion[comparacion['resultado'] == 'Coincide']
        faltantes = comparacion[comparacion['resultado'].isin(['Falta en dispersiones', 'Falta en dashboard'])]

        # === GUARDAR EXCEL DETALLADO ===
        salida = f"/content/drive/MyDrive/AUDITORIA/Resultado_{nombre}.xlsx"
        with pd.ExcelWriter(salida) as writer:
            coinciden.to_excel(writer, sheet_name='Coinciden', index=False)
            faltantes.to_excel(writer, sheet_name='Faltantes', index=False)

        print(f"✅ Resultado guardado en: {salida}")

        return resumen, comparacion

    except Exception as e:
        print(f"Error en {nombre}: {e}")
        return {}, pd.DataFrame()

# === EJECUTAR AUDITORÍA ===
resumen, comparacion_df = auditar_archivo(archivo_dashboard, df_dispersiones)

# === GUARDAR RESUMEN ===
df_resumen = pd.DataFrame([resumen])
ruta_resumen = '/content/drive/MyDrive/AUDITORIA/Resumen_Auditoria_Dashboard.xlsx'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columns in df_dispersiones after stripping: Index(['ID', 'Cus', 'Número Referencia', 'Medio Pago', 'Unnamed: 4',
       'Nombre Tramite', 'Número Factura', 'Documento', 'Entidad Bancaria',
       'Estado Transacción', 'VALOR', 'Fecha Transacción', 'Hora',
       'Día Hábil de Dispersión', 'RETENCION', 'Medio de Pago', 'DISPERSION',
       'ENTIDAD'],
      dtype='object')

🔍 Auditando: DASHBOARD AGOSTO 2025
✅ Resultado guardado en: /content/drive/MyDrive/AUDITORIA/Resultado_DASHBOARD AGOSTO 2025.xlsx
